In [1]:
# Download the corresponding PyTorch Geometric module
%%capture
"""
Assign to TORCH with what you get from the cell above, E.g., export TORCH=1.13.1+cu113
"""
%env TORCH=2.1.0+cu118
!pip install torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-geometric

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_geometric
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.data import DataLoader
import torch.nn.functional as F

/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_scatter/_version_cuda.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:124: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_sparse/_version_cuda.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


In [3]:
# # Load the Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]


# Check if CUDA is available and use it
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the data to the device (GPU if available)
data = data.to(device)

Processing...
Done!


In [20]:
import pdb
import torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool, global_max_pool
import random
from torch_geometric.utils import to_dense_adj

    # adj = to_dense_adj(edge_index)[0]
    # # Add self-loops to the adjacency matrix
    # adj = adj + torch.eye(adj.size(0), device=adj.device)
    # aggregated_neighbors = torch.matmul(adj, x)

class GIN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GIN, self).__init__()
        self.layer1 = torch.nn.Linear(input_dim, hidden_dim)
        self.layer2 = torch.nn.Linear(hidden_dim, hidden_dim)
        self.lin = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        adj = to_dense_adj(edge_index, max_num_nodes=x.size(0))[0]
        # Move torch.eye to the same device as adj
        adj_i = adj + torch.eye(adj.size(0), device=adj.device)

        aggregated_neighbors0 =  torch.matmul(adj_i, x)
        x = self.layer1(aggregated_neighbors0)
        x = F.relu(x)

        aggregated_neighbors1 = torch.matmul(adj_i, x)
        x = self.layer2(aggregated_neighbors1)
        x = F.relu(x)

        x = self.lin(x)
        return x

    def get_ebd(self, x, edge_index):
        adj = to_dense_adj(edge_index, max_num_nodes=x.size(0))[0]
        # Move torch.eye to the same device as adj
        adj_i = adj + torch.eye(adj.size(0), device=adj.device)

        aggregated_neighbors0 =  torch.matmul(adj_i, x)
        x = self.layer1(aggregated_neighbors0)
        x = F.relu(x)

        aggregated_neighbors1 = torch.matmul(adj_i, x)
        x = self.layer2(aggregated_neighbors1)
        x = F.relu(x)
        return x


In [21]:
# Training the model
def train(model, data, optimizer, criterion, epochs=200):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        # Get the node features (data.x) and edge indices (data.edge_index)
        out = model(data.x, data.edge_index)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

        if epoch % 10 == 0:
            print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}')

# Evaluate the model
def evaluate(model, data):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        pred = out.argmax(dim=1)
        correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
        acc = correct / data.test_mask.sum()
        return acc.item()

from sklearn.metrics import f1_score
import numpy as np
# Replace the original `test()` function
def evaluate_dice(model, data):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        pred = out.argmax(dim=1)

        y_true = data.y[data.test_mask].cpu().numpy()
        y_pred = pred[data.test_mask].cpu().numpy()

        # Dice score is equivalent to macro F1 score for multiclass
        dice = f1_score(y_true, y_pred, average='macro')
        return dice

In [22]:
# Model hyperparameters
in_channels = dataset.num_node_features  # 1433 (Cora input feature size)
hidden_channels = 64
out_channels = dataset.num_classes  # 7 (Cora has 7 classes)

# Initialize the model
model = GIN(in_channels, hidden_channels, out_channels).to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)


In [23]:
# Train the model
train(model, data, optimizer, criterion, epochs=50)

# Evaluate the model
acc = evaluate(model, data)
print(f'Test accuracy: {acc:.4f}')

dice = evaluate_dice(model, data)
print(f'Test dice: {dice:.4f}')

Epoch 1/50, Loss: 2.1057
Epoch 11/50, Loss: 0.3252
Epoch 21/50, Loss: 0.0227
Epoch 31/50, Loss: 0.0010
Epoch 41/50, Loss: 0.0002
Test accuracy: 0.7390
Test dice: 0.7439


In [24]:
from torch_geometric.utils import to_dense_adj
# Node features (X)
x = data.x  # Shape: [num_nodes, num_features]
# Adjacency matrix (A)
A = to_dense_adj(data.edge_index)[0]  # Shape: [num_nodes, num_nodes]
K = (data.x) @ (data.x).T
m = 1000

In [ ]:
from Perturbe_Algs import Attacks


num_rep = 5
list_ebd_random = []
list_ebd_pgdavg = []
list_ebd_pgdwst = []


list_acc_random = []
list_acc_pgdavg = []
list_acc_pgdwst = []


from copy import deepcopy

attack_instance = Attacks(A,K,m, alpha = 50, max_iter=200, filter = 'adj').to(device)

pgdavg_dataset = deepcopy(data)
pgdwst_dataset = deepcopy(data)

A_pgd_avg = attack_instance.Prob_PGD().clone().detach()
A_pgd_wst = attack_instance.Wst_PGD().clone().detach()

pgdavg_dataset.edge_index = torch.tensor(A_pgd_avg).nonzero(as_tuple=False).t().contiguous()
pgdwst_dataset.edge_index = torch.tensor(A_pgd_wst).nonzero(as_tuple=False).t().contiguous()

for i in range(num_rep):


  A_random = attack_instance.randomAttack().clone().detach()

  random_dataset = deepcopy(data)
  random_dataset.edge_index = torch.tensor(A_random).nonzero(as_tuple=False).t().contiguous()

  acc = evaluate(model, data)
  dice = evaluate_dice(model, data)

  list_acc_random.append(acc_random)
  list_acc_pgdavg.append(acc_pgdavg)
  list_acc_pgdwst.append(acc_pgdwst)


  ebd = model.get_ebd(data.x, data.edge_index)
  ebd_random = model.get_ebd(random_dataset.x, random_dataset.edge_index)
  ebd_pgdavg = model.get_ebd(pgdavg_dataset.x, pgdavg_dataset.edge_index)
  ebd_pgdwst = model.get_ebd(pgdwst_dataset.x, pgdwst_dataset.edge_index)


  p_ebd_random = torch.norm(ebd - ebd_random, p='fro')
  p_ebd_pgdavg = torch.norm(ebd - ebd_pgdavg, p='fro')
  p_ebd_pgdwst = torch.norm(ebd - ebd_pgdwst, p='fro')

  list_ebd_random.append(p_ebd_random.item())
  list_ebd_pgdavg.append(p_ebd_pgdavg.item())
  list_ebd_pgdwst.append(p_ebd_pgdwst.item())



In [28]:
print(np.mean(list_acc_random), np.std(list_acc_random))
print(np.mean(list_acc_pgdwst), np.std(list_acc_pgdwst))
print(np.mean(list_acc_pgdavg), np.std(list_acc_pgdavg))


0.7085999965667724 0.007002849813478941
0.718999981880188 0.0
0.5339999794960022 0.0


In [29]:
print(np.mean(list_ebd_random), np.std(list_ebd_random))
print(np.mean(list_ebd_pgdwst), np.std(list_ebd_pgdwst))
print(np.mean(list_ebd_pgdavg), np.std(list_ebd_pgdavg))


1087.58154296875 16.359313522959333
17718.919921875 0.0
54653.91015625 0.0
